In [2]:
import numpy as np
import pandas as pd
from collections import Counter

data = pd.read_csv('dataset.csv')

X = data.drop('label', axis=1).values
y = data['label'].values

def train_test_split_custom(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    
    X = X[indices]
    y = y[indices]
    
    split_index = int((1 - test_size) * len(X))
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split_custom(X, y, test_size=0.2, random_state=42)

class CustomKNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = [self._predict(x) for x in X_test]
        return np.array(predictions)

    def _predict(self, x):
        distances = [np.sqrt(np.sum((x - x_train)**2)) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_neighbor_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_neighbor_labels).most_common(1)
        return most_common[0][0]

knn = CustomKNN(k=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 92.12%


In [3]:
import joblib

joblib.dump(knn, 'model/digit_recognizer_3')


['model/digit_recognizer_3']

In [ ]:
import joblib
import cv2
import numpy as np
import pyscreenshot as ImageGrab
import time

model = joblib.load("model/digit_recognizer_3")
image_folder = "./img/"

while True:

    img = ImageGrab.grab(bbox=(60, 470, 400, 870))
    img.save(image_folder + "img.png")
    

    im = cv2.imread(image_folder + "img.png")
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gray = cv2.GaussianBlur(im_gray, (15, 15), 0)

    _, im_th = cv2.threshold(im_gray, 100, 255, cv2.THRESH_BINARY)
    roi = cv2.resize(im_th, (28, 28), interpolation=cv2.INTER_AREA)
 
    roi = roi / 255.0
    X = roi.flatten()  

    predictions = model.predict([X])
    print("Prediction:", predictions[0])

    cv2.putText(im, f"Prediction: {predictions[0]}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Result", im)

    time.sleep(5)

    if cv2.waitKey(1) == 13:
        break

cv2.destroyAllWindows()


Prediction: 3
Prediction: 3


KeyboardInterrupt: 

: 